In [1]:
import pandas as pd

file_path = "netflix_titles.csv"

pd.options.mode.chained_assignment = None

data = pd.read_csv(file_path,sep=",",header=0,encoding="utf-8",index_col=False)
data = data.dropna()

def create_new_csv_no(name, data):
    data.to_csv(name, sep=",", encoding="utf-8",index=False)


In [2]:
#Show.csv
show = data[['show_id','title', 'description','duration','type','release_year','date_added']]
show['type'] = show['type'].replace('TV Show', '1')
show['type'] = show['type'].replace('Movie', '0')
show['type'] = show['type'].astype(int)
show['duration'] = show['duration'].str.extract('(\d+)', expand=False).astype(int)
create_new_csv_no("show.csv", show)

In [3]:
#Movie.csv
movie = data[['show_id','title', 'description','duration','type','release_year','date_added']]
movie= movie[movie['type'] == 'Movie']
movie = movie.drop(columns=['type'])
movie['duration'] = movie['duration'].str.extract('(\d+)', expand=False).astype(int)
create_new_csv_no("movie.csv", movie)

In [4]:
#Series.csv
series = data[['show_id','title', 'description','duration','type','release_year','date_added']]
series= series[series['type'] == 'TV Show']
series = series.drop(columns=['type'])
series['duration'] = series['duration'].str.extract('(\d+)', expand=False).astype(int)
create_new_csv_no("series.csv", series)


In [5]:
#person.csv
person1 = pd.DataFrame(data['cast'].str.split(',').explode().str.strip())
person1.columns = ['person']
person2 = pd.DataFrame(data['director'].str.split(',').explode().str.strip())
person2.columns = ['person']

person = pd.concat([person1, person2])
person.drop_duplicates(subset="person", keep='first', inplace=True)
person['no_id'] = person.reset_index().index
create_new_csv_no("person.csv", person)

In [6]:
#Country.csv
country = pd.DataFrame(data['country'].str.split(',').explode().str.strip())
country.drop_duplicates(subset = 'country', keep = 'first', inplace = True)
country['no_country'] = country.reset_index().index
create_new_csv_no("country.csv", country)

In [7]:
#Rating.csv
rating = pd.DataFrame(data['rating'].str.split(',').explode().str.strip())
rating.drop_duplicates(subset = 'rating', keep = 'first', inplace = True)
rating['no_rating'] = rating.reset_index().index
create_new_csv_no("rating.csv", rating)

In [8]:
#Genre.csv
genre = pd.DataFrame(data['listed_in'].str.split(',').explode().str.strip())
genre.drop_duplicates(subset = 'listed_in', keep = 'first', inplace = True)
genre['no_genre'] = genre.reset_index().index

create_new_csv_no("genre.csv", genre)

In [9]:
#Actor.csv
cast = data[['show_id','cast']]
# Decompose the column "cast" into different names of persons with show_id
cast['cast'] = cast['cast'].str.split(', ')
cast= cast.explode('cast')

person= pd.read_csv("person.csv",sep=",",header=0,encoding="utf-8",index_col=False)

cast = cast.merge(person, left_on='cast', right_on='person', how='left')
cast = cast[['show_id', 'no_id']]
cast.columns = ['show_id', 'cast_id']
cast['cast_id'] = cast['cast_id'].fillna(0).astype(int)
cast = cast.drop_duplicates()
create_new_csv_no("actor.csv", cast)




In [10]:
#Director.csv

director = data[['show_id','director']]

director['director'] = director['director'].str.split(', ')
director= director.explode('director')

director = director.merge(person, left_on='director', right_on='person', how='left')
director = director[['show_id', 'no_id']]
director.columns = ['show_id', 'director_id']
director = director.drop_duplicates()
create_new_csv_no("director.csv", director)

In [11]:
#Produced.csv

produced = data[['show_id','country']]
produced['country'] = produced['country'].str.split(', ')
produced= produced.explode('country')

country= pd.read_csv("country.csv",sep=",",header=0,encoding="utf-8",index_col=False)

produced = produced.merge(country, left_on='country', right_on='country', how='left')
produced = produced[['show_id', 'no_country']]
produced.columns = ['show_id', 'country_id']
create_new_csv_no("produced.csv", produced)

In [12]:
#Belong.csv

belong = data[['show_id','listed_in']]

belong['listed_in'] = belong['listed_in'].str.split(', ')
belong= belong.explode('listed_in')

genre= pd.read_csv("genre.csv",sep=",",header=0,encoding="utf-8",index_col=False)

belong = belong.merge(genre, left_on='listed_in', right_on='listed_in', how='left')
belong = belong[['show_id', 'no_genre']]
belong.columns = ['show_id', 'genre_id']
create_new_csv_no("belong.csv", belong)

In [13]:
#Person.csv with lastname and firstname

person['firstname'] = person['person'].str.split(' ').str[0]
person['lastname'] = person['person'].str.split(' ').str[-1]
person.drop(columns=['person'], inplace=True)
create_new_csv_no("person.csv", person)

In [14]:
#Rated.csv 

find_rated = data[['show_id','rating']]

rating = pd.read_csv("rating.csv",sep=",",header=0,encoding="utf-8",index_col=False)
rated = find_rated.merge(rating, left_on='rating', right_on='rating', how='left')
rated = rated[['show_id', 'no_rating']]
rated.columns = ['show_id', 'rating_id']
create_new_csv_no("rated.csv", rated)
